Should the most important meal of the day be yummy or nutritious? Can't it be both?

In [49]:
import numpy as np
import pandas as pd
import plotly #I will only be using plotly express to visualise the data
import plotly.express as px 
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True) 
import scipy.stats as stats

In [45]:
df=pd.read_csv("cereal.csv")
df.rename(columns=lambda x:x.title(),inplace=True)
df.rename(columns={"Mfr":"Manufacturer","Potass":"Potassium","Carbo":"Carbohydrates"},inplace=True)
df.head()

,Name,Manufacturer,Type,Calories,Protein,Fat,Sodium,Fiber,Carbohydrates,Sugars,Potassium,Vitamins,Shelf,Weight,Cups,Rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


A survey of 80 cereals was conducted to include information on:
- *Name*: cereal name
- *Manufacturer*: cereal manufacturer
    - *A* = American Home Food Products
    - *G* = General Mills
    - *K* = Kelloggs
    - *N* = Nabisco
    - *P* = Post
    - *Q* = Quaker Oats
    - *R* = Ralston Purina
- *Type*:
    - *C* = cold
    - *H* = hot
- *Calories*: calories per serving
- *Protein*: grams of protein
- *Fat*: grams of fat
- *Sodium*: milligrams of sodium
- *Fiber*: grams of dietary fiber
- *Carbohydrates*: grams of complex carbohydrates
- *Sugars*: grams of sugars
- *Potassium*: milligrams of potassium
- *Vitamins*: percentage (0, 25, or 100) of vitamins and minerals recommended by the FDA
- *Shelf*: location of the cereal on the display shelf counting from the floor (1, 2, or 3)
- *Weight*: weight in ounces of one serving
- *Cups*: number of cups in one serving
- *Rating*: rating of the cereal

You should note that the survey was conducted in the early 90's so you most probably won't be able to find many of the cereals available to us today. What we can do is determine what people were eating almost 2 decades ago!

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           77 non-null     object 
 1   Manufacturer   77 non-null     object 
 2   Type           77 non-null     object 
 3   Calories       77 non-null     int64  
 4   Protein        77 non-null     int64  
 5   Fat            77 non-null     int64  
 6   Sodium         77 non-null     int64  
 7   Fiber          77 non-null     float64
 8   Carbohydrates  77 non-null     float64
 9   Sugars         77 non-null     int64  
 10  Potassium      77 non-null     int64  
 11  Vitamins       77 non-null     int64  
 12  Shelf          77 non-null     int64  
 13  Weight         77 non-null     float64
 14  Cups           77 non-null     float64
 15  Rating         77 non-null     float64
dtypes: float64(5), int64(8), object(3)
memory usage: 9.8+ KB


No null values - good start.

In [10]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Name,77,77,Cream of Wheat (Quick),1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Manufacturer,77,7,K,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,77,2,C,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Calories,77,NaN,NaN,NaN,106.883,19.4841,50,100,110,110,160
Protein,77,NaN,NaN,NaN,2.54545,1.09479,1,2,3,3,6
Fat,77,NaN,NaN,NaN,1.01299,1.00647,0,0,1,2,5
Sodium,77,NaN,NaN,NaN,159.675,83.8323,0,130,180,210,320
Fiber,77,NaN,NaN,NaN,2.15195,2.38336,0,1,2,3,14
Carbohydrates,77,NaN,NaN,NaN,14.5974,4.27896,-1,12,14,17,23
Sugars,77,NaN,NaN,NaN,6.92208,4.44489,-1,3,7,11,15


However it is not possible to have negative *carbohydrates*, *sugars* or *potassium*, instead I will change these to 0:

In [11]:
df.loc[(df["Carbohydrates"]<0)|(df["Sugars"]<0)|(df["Potassium"]<0)]=df.loc[(df["Carbohydrates"]<0)|(df["Sugars"]<0)|(df["Potassium"]<0)].replace(-1,0)

In [48]:
px.bar(df.sort_values(by="Rating",ascending=True),x="Rating",y="Name",
      labels={"Name":"","Rating":"Rating (%)"},hover_name="Name",hover_data={"Name":False},
      color="Rating",color_continuous_scale="tealgrn",template="plotly")

The favourite cereal is *All-Bran with Extra Fibre* (so healthy!), and the least favourite is actually one of my favourites *Cap'n'Crunch*..

In [13]:
df["Manufacturer"].unique()

array(['N', 'Q', 'K', 'R', 'G', 'P', 'A'], dtype=object)

In [14]:
df["Type"].unique()

array(['C', 'H'], dtype=object)

In [50]:
px.sunburst(df,path=["Manufacturer","Type"])

Manufacturers *G* (General Mills) and *K* (Kelloggs) each supply more than a quarter of the cereals in the pool; and majority of the cereals are *C* (cold). Only three manufacuturers produce *H* (hot) cereals - *Q* (Quaker Oats), *N* (Nabisco) and *A* (American Home Food Products).

In [16]:
px.box(df,x="Manufacturer",y="Rating",labels={"Mfr":"Manufacturer","Rating":"Rating (%)"},
       title="Rating Distribution",color="Manufacturer")

In [17]:
df[["Name","Rating"]].loc[df["Manufacturer"]=="K"].sort_values(by="Rating",ascending=False).head(1)

,Name,Rating
3,All-Bran with Extra Fiber,93.704912


*N* (Nabisco) performs the best out of all the manufacturers, followed by *A* (American Home Food Products) but American Home Food Products only has one cereal. *Q* (Quaker Oats) has the largest rating distribution, and *K* (Kelloggs) has one high performing outlier cereal, *All-Bran with Extra Fibre*.

In [18]:
manu=df.drop(["Name","Type","Rating","Shelf","Weight","Cups"],axis=1).groupby(["Manufacturer"]).mean()

px.bar(manu,x=manu.index,y=["Calories","Protein","Fat","Sodium","Fiber","Carbohydrates","Sugars","Potassium","Vitamins"])

> double click on the legend to isolate the variable

*A* (American Home Food Products)'s cereal has viturally no *sodium* or *fibre* but the most *protein* compared to the other manufacturers. *Q* (Quaker Oats) has the most *fat*, and *P* (Post) the most *sugars*. 

In [19]:
px.histogram(manu,nbins=30,range_x=[0,200])

> double click on the legend to isolate the variable

In [20]:
sstd=[stats.tstd(df[i]) for i in manu.columns]
sskew=[stats.skew(df[i]) for i in manu.columns]
skurtosis=[stats.kurtosis(df[i]) for i in manu.columns]

stable=pd.DataFrame({"Standard Deviation":sstd,"Skew":sskew,"Kurtosis":skurtosis},index=manu.columns)
stable.style.background_gradient(cmap="BuPu")

,Standard Deviation,Skew,Kurtosis
Calories,19.484119,-0.436683,2.142090
Protein,1.094790,0.731221,1.032193
Fat,1.006473,1.143151,1.837354
Sodium,83.832295,-0.564435,-0.400151
Fiber,2.383364,2.384046,8.019160
Carbohydrates,4.232257,-0.343613,0.788007
Sugars,4.422840,0.050920,-1.166392
Potassium,71.251147,1.327448,1.697380
Vitamins,22.342523,2.415447,5.781316


*Sodium* followed by *potassium* has the largest spread; *vitamins* and *fibre* are the most skewed and not normally distributed; and again *fibre* and *vitamins* have the highest kurtosis.

In [21]:
px.imshow(df.corr(),color_continuous_scale="tealrose",color_continuous_midpoint=0)

In the data:
- The strongest positive correlation is between *fiber* and *potassium*;
- *Calories* have notable positive correlations with *weight*, *sugars* and *fat*, which makes nutritional sense; and
- Strong negative correlations between *sugars* and *rating*, and *calories* and *rating*.

In [22]:
px.scatter(df,x="Calories",y="Rating",trendline="ols",color_discrete_sequence=["gold"],
           labels={"Calories":"Calories per Serve","Rating":"Rating (%)"},
           hover_name="Name",hover_data={"Rating":":.2f"},marginal_x="histogram",marginal_y="box")

As we have discovered above, *calories* have strong positive correlations with *weight*, *sugars* and *fat* thus is can be safe to assume that these calorific cereals are generally more unhealthy, and also perform worse than their healthier counterparts.

In [23]:
px.scatter(df,x="Sugars",y="Fat",trendline="ols",trendline_color_override="mediumseagreen",
           color_continuous_scale="picnic",opacity=0.3,size="Calories",size_max=35,color="Calories",
           labels={"Sugars":"Sugar (g) per Serve","Fat":"Fat (g) per Serve","Calories":"Calories per Serve"},
           hover_name="Name")

Although individually *sugars* and *fat* have a strong positive correlation with *calories*, together *sugars* and *fat*  only have a light positive correlation with one another.

In [24]:
px.scatter(df,x=["Fiber","Protein","Potassium"],y="Rating",trendline="ols",
           labels={"Rating":"Rating (%)"},
           hover_name="Name",hover_data={"Rating":":.2f"})

*Fibre*, *protein* and *potassium* are the most correlated variables in decreasing order to a cereal's *rating*. Seems like people are opting for healthier cereals.

Given a completely new cereal that you have never tried before, would it be possible to determine its rating?

1. Clean the dataset to get rid of any categorical variables (i.e. *Manufacturer* and *Type*) by getting dummies and dropping the original columns:

In [25]:
df=pd.concat([df,pd.get_dummies(df["Manufacturer"])],axis=1)

In [26]:
df=pd.concat([df,pd.get_dummies(df["Type"])["C"]],axis=1)

In [27]:
df.rename(columns={"C":"Cold"},inplace=True)

In [28]:
df.drop(["Manufacturer","Type"],axis=1,inplace=True)

2. Also drop any unnecessary variables (i.e. *name* and *shelf*):

In [29]:
df.drop(["Name","Shelf"],axis=1,inplace=True)

3. Split the data into the training and testing set:

In [30]:
from sklearn.model_selection import train_test_split

x=df.drop(["Rating"],axis=1)
y=df["Rating"]

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=7)

In [31]:
print("x train shape",x_train.shape,"and y train shape",y_train.shape)

x train shape (57, 19) and y train shape (57,)


In [32]:
print("x test shape",x_test.shape,"and y test shape",y_test.shape)

x test shape (20, 19) and y test shape (20,)


4. Scale the data to ensure columns with larger numbers don't give a higher weighting:

In [33]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

5. Create, fit and predict the model:

In [34]:
from sklearn.neighbors import KNeighborsRegressor

model=KNeighborsRegressor(n_neighbors=5)
model.fit(x_train,y_train)
y_predict=model.predict(x_test)

6. Evaluate the model:

In [35]:
model.score(x_test,y_test)

0.718367798190463

In [36]:
results=pd.DataFrame({"Actual":y_test,"Predicted":y_predict})

px.histogram(results,x=["Actual","Predicted"],nbins=20,range_x=[15,65],opacity=0.3,marginal="rug",
            title="Distribution of the Acutal and Predicted Values")

In [37]:
a=results["Actual"]
b=results["Predicted"]

stats.ttest_ind(a,b)

Ttest_indResult(statistic=0.4132021588854063, pvalue=0.6817810024407129)

In [38]:
smin=[stats.tmin(results[i]) for i in results.columns]
smax=[stats.tmax(results[i]) for i in results.columns]
smean=[stats.tmean(results[i]) for i in results.columns]
sstd=[stats.tstd(results[i]) for i in results.columns]
sskew=[stats.skew(results[i]) for i in results.columns]
skurtosis=[stats.kurtosis(results[i]) for i in results.columns]

stable=pd.DataFrame({"Min":smin,"Max":smax,"Mean":smean,"Standard Deviation":sstd,"Skew":sskew,"Kurtosis":skurtosis},index=results.columns)
stable.style.background_gradient(cmap="coolwarm")

,Min,Max,Mean,Standard Deviation,Skew,Kurtosis
Actual,19.823573,64.533816,41.297828,11.979021,0.283683,-0.400146
Predicted,26.107569,64.354052,39.867140,9.811869,0.855533,0.249976


The model is 72% accurate with the actual and predicted results not statistically different having a similar mean and standard deviation, the same maximum value and acceptable kurtosis. Where the acutal and predicted results differ is a larger predicted minimum value and a more skewed predicted distribution.

As with k-nearest neighbours we can choose the number of nearest neighbours the model should look for. Maybe we can improve the model's accuracy by testing the model for the most optimal number of neighbours!

In [39]:
neighbours=list(range(1,20))
score=[]

for i in range (1,20):
    model=KNeighborsRegressor(n_neighbors=i)
    model.fit(x_train,y_train)
    y_predict_i=model.predict(x_test)
    score.append(model.score(x_test,y_test))

neighbourlist=pd.DataFrame({"No of Neighbours":neighbours,"Score":score})

fig=px.line(neighbourlist,x="No of Neighbours",y="Score",range_y=[0.2,0.83],color_discrete_sequence=["slateblue"])
fig.add_annotation(x=5,y=0.73,
                   text="Optimal number of neighbours, which <br> was already implemented in the initial model!",
                   standoff=0,arrowsize=1,arrowwidth=1.5,arrowhead=2)

Our initial model already had the optimal number of neighbours..

But wait a minute!

In [40]:
x.shape[1]

19

In [41]:
x.columns

Index(['Calories', 'Protein', 'Fat', 'Sodium', 'Fiber', 'Carbohydrates',
       'Sugars', 'Potassium', 'Vitamins', 'Weight', 'Cups', 'A', 'G', 'K', 'N',
       'P', 'Q', 'R', 'Cold'],
      dtype='object')

Our model was taking these 19 variables as input. We had already previously dropped unnecessary columns *Name* and *Shelf* but I wonder what the predictive power of these remaining variables are and if they are all important..

In [42]:
from sklearn.feature_selection import SelectKBest,f_regression

selector=SelectKBest(f_regression,k="all")
selector.fit(x_train,y_train)

pvals=pd.DataFrame({"variable":x.columns,"P-value":selector.pvalues_})
def color_pvalues_red(val):
    color="red" if val>0.05 else "black"
    return "color:%s" % color
pvals.style.applymap(color_pvalues_red,subset=pd.IndexSlice[:,"P-value"])

,variable,P-value
0,Calories,0.000000
1,Protein,0.000154
2,Fat,0.000908
3,Sodium,0.001132
4,Fiber,0.000000
5,Carbohydrates,0.921237
6,Sugars,0.000000
7,Potassium,0.000266
8,Vitamins,0.081051
9,Weight,0.022149


So it seems *carbohydrates*, *cups*, *A*, *K*, *P*, *Q*, *R* and *cold* do not have much predicitive power so we can remove these from the input and test if the model is any more accurate.

In [43]:
x_train=np.delete(x_train,[5,8,10,11,13,15,16,17,18],1)
x_test=np.delete(x_test,[5,8,10,11,13,15,16,17,18],1)

model=KNeighborsRegressor(n_neighbors=5)
model.fit(x_train,y_train)
y_predict=model.predict(x_test)

model.score(x_test,y_test)

0.9109939603399623

Yes we have improved the accuracy from it's initial 72% to 91% by removing columns with low predictive power!